# User Interface

User Interface is a node that is implemented in order to choose the velocity and orientation of the robot. It also allows to decide whether to start or stop a robot's motion at any given time. In addition, it can show the current state of the robot, such as the velocity, position, and orientation of the robot, as well as the goals that have been reached or cancelled.

## How to run the code

First, open a terminal, run the following code in the correct workspace in order to open the simulation environment:

- roslaunch rt2_ass2 sim.launch

Then, in another terminal, run the following code:
- jupyter noteboook

## Code for the User Interface:

First of all, headers and libraries are needed bedore writing the code. In this case, it will be:

In [1]:
import rospy
import time
import jupyros as jr
import matplotlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import math
import actionlib
import actionlib.msg

With Jupyter we can use all the python modules that we can import from our 'standard' python shell. As an
example, we can use matplotlib! By using the %matplotlib widget, we can have an interactive plot.

In [2]:
%matplotlib widget

from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from std_msgs.msg import String, Float64, Bool, Float32
from rt2_assignment2.srv import Command
from ipywidgets import interact,interactive,fixed,interact_manual
from geometry_msgs.msg import Twist

ModuleNotFoundError: No module named 'rt2_assignment2'

## Initialization of Node and Global Variables

In [ ]:
#Initialize the node. Topics: /vel and /cmd_vel
rospy.init_node('user_interface')
ui_client=rospy.ServiceProxy('/user_interface', Command)
pub_vel=rospy.Publisher('/vel', Twist, queue_size=1)
pub_cmd=rospy.Publisher('/cmd_vel', Twist, queue_size=1)



# robot behaviour
go_rand= True # wheater the robot is going random or not
vel=Twist() # the value of linear and angular velocities
cmd=Twist() # the command velocities I want to obtain

# linear and angualr velocity (requested and actual)
VelLin_s=0 
VelLin_a=0 
VelAng_s=0 
VelAng_a=0 
vel1_1=[] # array to save the linear velocities requested 
vel1_2=[] # array to save the actual linear velocities
vel2_1=[] # array to save the angular velocities requested
vel2_2=[] # array to save the actual angular velocities

#Time
initial_time=time.time() 
timeNow=0 
time1=[]  # array to save the times
time_to_reach=[] #array to save the times to reach a goal

x_data=[] # array to save the x coordinate of the robot
y_data=[] # array to save the y coordinate of the robot
dx=0.0 #x direction of the orientation
dy=1.0 #y direction of the orientation

stats = [0,0] #array to save the number of reached and cancelled goal
reached=0 #number of reached goals
cancelled=0 # number of cancelled goals




## Buttons 

Directly controlling the robot movements by using 5 Buttons: forward, backward, turn right, turn left, and stop, In addition, 2 more buttons to start or stop the random goal behavior of the robot.


In [4]:

# go straight forward
b1 = widgets.Button(description='up',
                    layout=widgets.Layout(width='auto', grid_area='up'),
                    style=widgets.ButtonStyle(button_color='yellow')
                   )

# go straight backwards
b2 = widgets.Button(description='down',
                    layout=widgets.Layout(width='auto', grid_area='down'),
                    style=widgets.ButtonStyle(button_color='yellow')
                   )

# turn right
b3 = widgets.Button(description='right',
                    layout=widgets.Layout(width='auto', grid_area='right'),
                    style=widgets.ButtonStyle(button_color='yellow')
                   )
# turn left
b4= widgets.Button(description='left',
                    layout=widgets.Layout(width='auto', grid_area='left'),
                    style=widgets.ButtonStyle(button_color='yellow')
                  )

# stop the robot
b5 = widgets.Button(description='stop',
                    layout=widgets.Layout(width='auto', grid_area='stop'),
                    style=widgets.ButtonStyle(button_color='orange')
                   )

# start random goal
b6 = widgets.Button(description='Start rand', 
                layout=widgets.Layout(widht='auto',grid_area='b6'),
                style=widgets.ButtonStyle(button_color='green')   
                )

# stop random goal
b7 = widgets.Button(description='Stop rand',
                    layout=widgets.Layout(width='auto', grid_area='b7'),
                    style=widgets.ButtonStyle(button_color='red')
                    )

## Functions associated to the buttons

In [ ]:
b1.on_click(up)
b2.on_click(down)
b3.on_click(right)
b4.on_click(left)
b5.on_click(stop_cmd)
b6.on_click(start_rand)
b7.on_click(stop_rand)


## Buttons and Sliders

In [ ]:

print("Linear Velocity")
# slider for the linear velocity
interact(setvellin,
        x=widgets.FloatSlider(min=0.0, max=1, step=0.01, value=0.3))

print("Angular Velocity")
#slider for the angular velocity
interact(setvelang,
        x=widgets.FloatSlider(min=0.0, max=1, step=0.01, value=0.3))

# box to set all of the buttons
widgets.GridBox(children=[b1, b2, b3, b4, b5, b6, b7],
                layout=widgets.Layout(width='80%',grid_template_rows='auto auto auto',
                grid_template_columns='16% 16% 16% 16% 16% 16% ',
                grid_template_areas='''". . . b7  . .""b1 b2 . . up . "" . .  . left down right "''')
               )

## Subscribers

In [ ]:
# this function transforms an angle in the interval [-pi, pi]
def normalize_angle(angle):
    if(math.fabs(angle) > math.pi):
        angle = angle - (2 * math.pi * angle) / (math.fabs(angle))
    return angle

#Convert a quaternion into euler angles (roll, pitch, yaw)
def euler_from_quaternion(x, y, z, w):
        #roll is rotation around x in radians (counterclockwise)
        #pitch is rotation around y in radians (counterclockwise)
        #yaw is rotation around z in radians (counterclockwise)
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians

# called when new data are published on the topic /odom
def odom_callback(msg):
    global timeNow, vel1_1, vel2_1, vel1_2, vel2_2, time1,dx,dy
    # retrieve how much time has passed between the init of the program
    timeNow= time.time()-initial_time
    # If I saved more than 500 data I delete the first one, for all arrays
    #I want arrays of maxium 500 elements to have a faster program
    if len(time1)>500:
        time1.pop(0)
        x_data.pop(0)
        y_data.pop(0)
        vel1_1.pop(0)
        vel1_2.pop(0)
        vel2_1.pop(0)
        vel2_2.pop(0)
    #add the new time 
    time1.append(timeNow)
    # add the new x position
    x_data.append(msg.pose.pose.position.x)
    # add the new y position
    y_data.append(msg.pose.pose.position.y)
    # add the new linear velocity actual
    vel1_2.append(msg.twist.twist.linear.x)
    # add the new angular velocity actual
    vel2_2.append(msg.twist.twist.angular.z)
    # add the new requested linear velocity
    vel1_1.append(VelLin_s)
    # add the new requested angular velocity
    vel2_1.append(VelAng_s)
    # Reading the quaternion to obtain the orientation of the robot
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    # converting in Euler angles
    euler = euler_from_quaternion(quaternion[0],quaternion[1],quaternion[2],quaternion[3])
    # obtaining the angle of rotation about the z axis, it's positive value is counterclock-wise
    yaw_ = euler[2]
    # normalizing the yaw angle
    normalized_yaw=normalize_angle(yaw_)
    #obtaining the x and y components of an array representing orientation
    dx=math.cos(normalized_yaw)
    dy=math.sin(normalized_yaw)
    
# called when new data are published on the topic /cmd_vel
def vel_callback(msg):
    global VelLin_s, VelAng_s
    # saving the values of the requested velocities in global variables
    VelLin_s=msg.linear.x
    VelAng_s=msg.angular.z
    
# called when new data are published on the topic /reach
def reach_callback(msg):
    global reached,cancelled,stats
    # If the message is True the goal was reached and I need to add 1 to the global variables reached
    if msg.data:
        reached+=1
    # If the message is Flse I need to add 1 to the cancelled variable
    else: 
        cancelled+=1
    # updating the vector storing the data    
    stats=[reached, cancelled]

# called when new data are published on the topic /time   
def time_callback(msg):
    global time_to_reach
    # I add the time to reach the goal to the vector storing those information
    time_to_reach.append(msg.data)    

# declaring all of the subscribers    
vel_sub= rospy.Subscriber('/cmd_vel',Twist, vel_callback)    
sub= rospy.Subscriber('/odom',Odometry, odom_callback)
reach_sub= rospy.Subscriber('/reach',Bool, reach_callback) 
time_sub= rospy.Subscriber('/time',Float32, time_callback)

## Velocity

In [ ]:
# setting the first plot
fig1,ax1=plt.subplots()
ax1.set_ylim((-2,2))
ax1.set_xlim((0,10))
lobj2 =ax1.plot([],[],lw=2, color='red',label='linear velocity actual')[0]
lobj1 =ax1.plot([],[], lw=2, color='green',label='linear velocity set')[0]
line1,=ax1.plot([],[], lw=2)
ax1.legend()
ax1.set_title("Linear Velocity")
ax1.set_xlabel('time')
lines1=[]
lines1.append(lobj1)
lines1.append(lobj2)

# setting the fsecond plot
fig2,ax2=plt.subplots()
ax2.set_ylim((-2,2))
ax2.set_xlim((0,10))
lobj2 =ax2.plot([],[],lw=2, color='red',label='angular velocity actual')[0]
lobj1 =ax2.plot([],[], lw=2, color='green',label='angular velocity set')[0]
line2,=ax2.plot([],[], lw=2)
ax2.legend()
ax2.set_title("Angular Velocity")
ax2.set_xlabel('time')
lines2=[]
lines2.append(lobj1)
lines2.append(lobj2)

# function called to initialize the plot
def init1():
    line1.set_data([],[])

# function called to initialize the plot    
def init2():
    line2.set_data([],[])

# function called to update periodically the plot
def animate1(i):
    ax1.set_xlim((timeNow-5,timeNow+5))
    xlist = [time1, time1]
    ylist = [vel1_1, vel1_2]
    #for index in range(0,1):
    for lnum1,line1 in enumerate(lines1):
        line1.set_data(xlist[lnum1], ylist[lnum1]) # set data for each line separately.
    return (lines1)

# function called to update periodically the plot
def animate2(i):
    ax2.set_xlim((timeNow-5,timeNow+5))
    xlist = [time1, time1]
    ylist = [vel2_1, vel2_2]
    #for index in range(0,1):
    for lnum2,line2 in enumerate(lines2):
        line2.set_data(xlist[lnum2], ylist[lnum2]) # set data for each line separately.
    return (lines2)

ani1=animation.FuncAnimation(fig1, animate1, init_func=init1, frames=100, interval=100, blit=True)
ani2=animation.FuncAnimation(fig2, animate2, init_func=init2, frames=100, interval=100, blit=True)

## Graphs for Velocity, Position, Time, and Reaching or Cancelled Goals

In [ ]:
# For velocities:

# setting the first plot
fig1,ax1=plt.subplots()
ax1.set_ylim((-2,2))
ax1.set_xlim((0,10))
lobj2 =ax1.plot([],[],lw=2, color='red',label='linear velocity actual')[0]
lobj1 =ax1.plot([],[], lw=2, color='green',label='linear velocity set')[0]
line1,=ax1.plot([],[], lw=2)
ax1.legend()
ax1.set_title("Linear Velocity")
ax1.set_xlabel('time')
lines1=[]
lines1.append(lobj1)
lines1.append(lobj2)

# setting the fsecond plot
fig2,ax2=plt.subplots()
ax2.set_ylim((-2,2))
ax2.set_xlim((0,10))
lobj2 =ax2.plot([],[],lw=2, color='red',label='angular velocity actual')[0]
lobj1 =ax2.plot([],[], lw=2, color='green',label='angular velocity set')[0]
line2,=ax2.plot([],[], lw=2)
ax2.legend()
ax2.set_title("Angular Velocity")
ax2.set_xlabel('time')
lines2=[]
lines2.append(lobj1)
lines2.append(lobj2)

# function called to initialize the plot
def init1():
    line1.set_data([],[])

# function called to initialize the plot    
def init2():
    line2.set_data([],[])

# function called to update periodically the plot
def animate1(i):
    ax1.set_xlim((timeNow-5,timeNow+5))
    xlist = [time1, time1]
    ylist = [vel1_1, vel1_2]
    #for index in range(0,1):
    for lnum1,line1 in enumerate(lines1):
        line1.set_data(xlist[lnum1], ylist[lnum1]) # set data for each line separately.
    return (lines1)

# function called to update periodically the plot
def animate2(i):
    ax2.set_xlim((timeNow-5,timeNow+5))
    xlist = [time1, time1]
    ylist = [vel2_1, vel2_2]
    #for index in range(0,1):
    for lnum2,line2 in enumerate(lines2):
        line2.set_data(xlist[lnum2], ylist[lnum2]) # set data for each line separately.
    return (lines2)

ani1=animation.FuncAnimation(fig1, animate1, init_func=init1, frames=100, interval=100, blit=True)
ani2=animation.FuncAnimation(fig2, animate2, init_func=init2, frames=100, interval=100, blit=True)

In [ ]:
# For Position and Orientation of the robot:

fig3,ax3=plt.subplots()
fig4,ax4=plt.subplots()
ax3.set_xlim((-10,10))
ax3.set_ylim((-10,10))
line3,=ax3.plot([],[], lw=2, color='red', label='position')
ax3.legend()
ax3.set_title("Position")
ax4.quiver(0,0,dx,dy,label='orientation')
ax4.legend()
ax4.set_title("Orientation")

# function called to initialize the plot
def init3():
    line3.set_data([],[])
    
# function called to initialize the plot
def init4():
    ax4.quiver(0,0,dx,dy,label='orientation')
    
# function called to update periodically the plot 
def animate3(i):
    global x_data, y_data
    line3.set_data(x_data, y_data)
    return (line3,)

# function called to update periodically the plot 
def animate6(i):
    ax4.clear()
    ax4.quiver(0,0,dx,dy,label='orientation')
    ax4.legend()
    ax4.set_title("Orientation")
    return (line4,)

ani3=animation.FuncAnimation(fig3, animate3, init_func=init3, frames=100, interval=100, blit=True)
ani4=animation.FuncAnimation(fig4, animate6, frames=1, interval=100, blit=True)


In [ ]:
# For Time:

fig5,ax5= plt.subplots()
ax5.hist(time_to_reach, 10, label='time to reach goal', color='red',lw=10)
ax5.legend()
ax5.set_title("Time to reach Goal")
ax5.set_xlabel('time (s)')

# function called to initialize the plot
def init5():
    ax5.hist(time_to_reach, 10, label='time to reach goal', color='red',lw=10)

# function called to update periodically the plot    
def animate5(i):
    global time_to_reach, ax5
    ax5.hist(time_to_reach,10, color='red',lw=10)
    
ani5=animation.FuncAnimation(fig5, animate5,init_func=init5, frames=100, interval=100, blit=True)

In [ ]:
# For Reaching and Cancelled Goals:

# setting the figure properties
fig6,ax6 = plt.subplots()
legend = ['reached', 'cancelled']
ax6.bar(legend,stats,color = 'r', width = 0.5)
ax6.set_title("Reached and Cancelled goal")

# function called to initialize the plot
def init6():
    ax6.bar(legend,stats,color = 'r', width = 0.5)

# function called to update periodically the plot
def animate6(i):
    global stats, ax6
    ax6.bar(legend,stats,color = 'r', width = 0.5)
    
ani6=animation.FuncAnimation(fig6, animate6, init_func=init6, frames=100, interval=100, blit=True)

